### Load and preprocessing images  

In [1]:
import os
import cv2
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# The address of the folder where the images are located
path = "C:\\Users\\Amin\\Desktop\\Mask"

images = []
labels = []
image_list = os.listdir(path)
image_size = 64
for image_name in image_list: 
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (image_size,image_size)) 
    image = image.astype('float32')         
    image = image / 255.0                           
    images.append(image)
    labels.append(image_name[-5])
np.unique(labels)

array(['0', '1'], dtype='<U1')

### Separate x and y data

In [2]:
images = np.array(images)
labels = to_categorical(labels, num_classes=2)

print("images: ",images.shape)
print("Labels: ",labels.shape)
print(labels)

images:  (1238, 64, 64)
Labels:  (1238, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


### Separate train and test data

In [3]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=3)

print("train_images: ",train_images.shape)
print("test_images:  ",test_images.shape)
print("train_labels: ",train_labels.shape)
print("test_labels:  ",test_labels.shape)

train_images:  (990, 64, 64)
test_images:   (248, 64, 64)
train_labels:  (990, 2)
test_labels:   (248, 2)


### Modeling

In [4]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD 

# create model
model = Sequential()
model.add(Conv2D(32 , kernel_size =(9,9) , activation='relu' , input_shape=(image_size,image_size,1) ))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64 , (9,9) , activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128 , (9,9) , activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# model.add(Conv2D(256 , (3,3) , activation='relu'))
# model.add(MaxPooling2D((2,2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(0.001),
              metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 56, 56, 32)        2624      
                                                                 
 max_pooling2d (MaxPooling2  (None, 28, 28, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        165952    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 64)        0

In [5]:
cnn = model.fit(train_images, train_labels, epochs=5,
                    validation_data=(train_images, train_labels))

Epoch 1/5
31/31 [==============================] - 3s 85ms/step - loss: 0.6160 - accuracy: 0.7596 - val_loss: 0.6143 - val_accuracy: 0.7697
Epoch 2/5
31/31 [==============================] - 3s 84ms/step - loss: 0.5744 - accuracy: 0.7687 - val_loss: 0.5804 - val_accuracy: 0.7697
Epoch 3/5
31/31 [==============================] - 3s 84ms/step - loss: 0.5544 - accuracy: 0.7697 - val_loss: 0.5637 - val_accuracy: 0.7697
Epoch 4/5
31/31 [==============================] - 3s 84ms/step - loss: 0.5488 - accuracy: 0.7697 - val_loss: 0.5567 - val_accuracy: 0.7697
Epoch 5/5
31/31 [==============================] - 3s 83ms/step - loss: 0.5499 - accuracy: 0.7697 - val_loss: 0.5549 - val_accuracy: 0.7697
